In [1]:
# from __future__ import absolute_import, division, print_function, unicode_literals

# from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Embedding, Dropout, BatchNormalization
# from tensorflow.keras.datasets import imdb
# from keras.datasets import boston_housing
import numpy as np
import tensorflow as tf
from tensorflow import keras

import os
import sys
sys.path.append('..')  
from GradPri_utils.utils import *
from tensorflow.keras.preprocessing import image

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
DATA_PATH_train = 'ants_bees/train'
file_name_train_ants = getfile_name(os.path.join(DATA_PATH_train, 'ants'))
file_name_train_bees = getfile_name(os.path.join(DATA_PATH_train, 'bees'))
input_size = 224 #224 for Vgg19 and ResNet101V2, 150 for Xception, 299 for InceptionV3

trainX = []
trainY = []
for i in range(len(file_name_train_ants)):
    img_path = os.path.join(DATA_PATH_train, 'ants', file_name_train_ants[i])
    img = image.load_img(img_path, target_size=(input_size, input_size))
    x_tmp = image.img_to_array(img)
    trainX.append(x_tmp)
    trainY.append(0)

for i in range(len(file_name_train_bees)):
    img_path = os.path.join(DATA_PATH_train, 'bees', file_name_train_bees[i])
    img = image.load_img(img_path, target_size=(input_size, input_size))
    x_tmp = image.img_to_array(img)
    trainX.append(x_tmp)
    trainY.append(1)

In [3]:
DATA_PATH_test = 'ants_bees/test'
file_name_test_ants = getfile_name(os.path.join(DATA_PATH_test, 'ants'))
file_name_test_bees = getfile_name(os.path.join(DATA_PATH_test, 'bees'))

testX = []
testY = []
for i in range(len(file_name_test_ants)):
    img_path = os.path.join(DATA_PATH_test, 'ants', file_name_test_ants[i])
    img = image.load_img(img_path, target_size=(input_size, input_size))
    x_tmp = image.img_to_array(img)
    testX.append(x_tmp)
    testY.append(0)

for i in range(len(file_name_test_bees)):
    img_path = os.path.join(DATA_PATH_test, 'bees', file_name_test_bees[i])
    img = image.load_img(img_path, target_size=(input_size, input_size))
    x_tmp = image.img_to_array(img)
    testX.append(x_tmp)
    testY.append(1)

In [4]:
trainX = np.array(trainX)
trainY = np.array(trainY)
testX = np.array(testX)
testY = np.array(testY)

In [5]:
print(np.shape(trainX), np.shape(trainY), np.shape(testX), np.shape(testY))

(244, 224, 224, 3) (244,) (153, 224, 224, 3) (153,)


In [21]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16

In [32]:
# # First, instantiate a base model with pre-trained weights.
# base_model = keras.applications.Xception(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(input_size, input_size, 3),
#     include_top=False)  # Do not include the ImageNet classifier at the top.

# base_model = keras.applications.InceptionV3(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(input_size, input_size, 3),
#     include_top=False)  # Do not include the ImageNet classifier at the top.

# base_model = keras.applications.ResNet101V2(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(input_size, input_size, 3),
#     include_top=False)  # Do not include the ImageNet classifier at the top.

base_model = ResNet50(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(32, 32, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [33]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_13[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [34]:
inputs = keras.Input(shape=(32, 32, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors

x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024)(x)
x = keras.layers.Dense(128)(x)
# x = keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
# x = keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
# x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
# x = keras.layers.Flatten()(x)
# x = keras.layers.Dense(1024, activation='relu')(x)
# x = keras.layers.Dense(128, activation='relu')(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(10)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 2048)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dense_14 (Dense)             (None, 128)               131200    
_________________________________________________________________
dense_15 (Dense)             (None, 10)                1290      
Total params: 25,818,378
Trainable params: 25,765,258
Non-trainable params: 53,120
__________________________________________